In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pickle

### Get a list of all guides

In [2]:
req = Request('https://guides.nyu.edu/sitemap.xml')
html_page = urlopen(req).read()
soup = BeautifulSoup(html_page, 'xml')
guide_urls = set()
for item in soup.find_all('loc'):
    guide_urls.add(item.text)
print("registered", len(guide_urls), "top level guide URLs")

registered 319 top level guide URLs


### For each guide, get a list of associated resources/pages

In [3]:
guides = dict()
urls_count = 0
skip_count = 0

content_hashes = set()

for guide_url in guide_urls:
    req = Request(guide_url)
    html_page = urlopen(req).read()
    soup = BeautifulSoup(html_page, 'html')
    _subguides_for_this_guide = set()
    for a in soup.find_all('a', href=True):
        if guide_url in a['href']:
            _content_hash = hash(soup)
            # if we've seen this page before, move on
            if _content_hash in content_hashes:
                #print(f"skipping {a['href']}")
                skip_count += 1
                continue
            else:
                content_hashes.add(_content_hash)
            _subguides_for_this_guide.add(a['href'])
            urls_count += 1
    guides[guide_url] = _subguides_for_this_guide
    
print("skipped", skip_count, "subject level guide URLs")
print("registered", urls_count, "subject level guide URLs")

skipped 2954 subject level guide URLs
registered 317 subject level guide URLs


### List the associated resources for the ai-tools guide

In [4]:
print(guides["https://guides.nyu.edu/ai-tools"])

{'https://guides.nyu.edu/ai-tools'}


### Save to a file

In [5]:
with open("guides.pickle", "wb") as f:
    pickle.dump(guides, f)